In [ ]:
%cd ..

In [ ]:
import pandas as pd
from bankroll.interface import *
from decimal import Decimal
from ib_insync import IB, util
import logging

In [ ]:
util.startLoop()

accounts = AccountAggregator.fromSettings(AccountAggregator.allSettings(config=loadConfig()), lenient=False)

In [ ]:
stockPositions = [p for p in accounts.positions() if isinstance(p.instrument, Stock) and p.quantity != 0]
stockPositions.sort(key=lambda p: p.instrument)

In [ ]:
values = liveValuesForPositions(stockPositions, marketDataProvider(accounts))
for p in stockPositions:
    if p in values:
        continue
    
    logging.warning(f'Missing market value for {p}')
    values[p] = Cash(currency = p.instrument.currency, quantity = Decimal(0))

In [ ]:
realizedBases = {p: realizedBasisForSymbol(p.instrument.symbol, accounts.activity()) for p in stockPositions}

In [ ]:
rows = {p.instrument.symbol: [
    values[p],
    p.costBasis,
    values[p] - p.costBasis,
    (values[p] - p.costBasis) / p.costBasis,
    realizedBases[p],
    values[p] - realizedBases[p],
    (values[p] - realizedBases[p]) / realizedBases[p],
] for p in stockPositions}

In [ ]:
def colorNegativeValue(val):
    color = 'red' if val < 0 else 'black'
    return 'color: %s' % color

In [ ]:
df = pd.DataFrame.from_dict(data=rows, orient='index', columns=[
    'Market value',
    'Cost basis',
    'Tax P/L',
    'Tax P/L %',
    'Realized basis',
    'Net P/L',
    'Net P/L %',
])

df.style.format({
    'Tax P/L %': '{:.2%}',
    'Net P/L %': '{:.2%}',
}).applymap(colorNegativeValue)